# Alzheimer’s Disease snRNA-seq Analysis (Entorhinal Cortex)

**Author:** Olivia Mohning  
**Repository:** `scrnaseq`  
**Notebook:** `notebooks/01_data_ingestion_and_eda.ipynb`  
**Created:** 2025-08-07  

This notebook uses single-nucleus RNA-sequencing (snRNA-seq) data from the entorhinal cortex of Alzheimer’s disease and control brains (dataset **GSE138852**) to perform exploratory data analysis in Python. The workflow focuses on reproducible data ingestion, cleaning, integration of metadata, and visualization using the pandas data stack.  

### Objectives
- Load and align single-nucleus RNA-seq count matrices with associated metadata  
- Inspect dataset structure and cell-type composition  
- Perform exploratory data analysis (EDA) of gene expression across cell types and disease conditions (AD vs Control)  
- Visualize expression of key genes (e.g., APOE, CLU, GFAP) to identify early transcriptional patterns related to neurodegeneration  

**Data Source:** GSE138852 – Human entorhinal cortex nuclei from aged Alzheimer’s disease and control individuals  
**Platform:** Illumina NextSeq 500  
**Notes:** Processed count and metadata files are stored in `data/GSE138852/` and loaded directly for analysis.  

### Import modules, suppress warnings

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module=r"louvain")
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc

### Auto-detect repo root by walking up from current dir

In [4]:
def find_repo_root(start = Path.cwd()):
    for p in [start] + list(start.parents):
        if (p / "notebooks").is_dir() and (p / "data").is_dir() and (p / "README.md").exists():
            return p
    raise FileNotFoundError("Could not locate repo root. Are you running inside the 'scrnaseq' repo?")

repo_root = find_repo_root()

### Define file paths for later use

In [5]:
data_dir = repo_root / "data"
src_dir  = data_dir / "raw" / "GSE138852"
proc_dir = data_dir / "processed"
fig_dir  = repo_root / "figures"
res_dir  = repo_root / "results"

### Make sure output directories exist

In [6]:
for d in (proc_dir, fig_dir, res_dir):
    d.mkdir(parents=True, exist_ok=True)

### Assign variables to file paths for raw data

In [7]:
counts_path = src_dir / "GSE138852_counts.csv"
meta_path   = src_dir / "GSE138852_covariates.csv"

### Sanity checks for repo root and file paths

In [8]:
print("Repo root:", repo_root)
print("Counts path:", counts_path.exists(), counts_path)
print("Meta path:",   meta_path.exists(),   meta_path)

Repo root: /Users/oliviamohning/Documents/ds-portfolio/scrnaseq
Counts path: True /Users/oliviamohning/Documents/ds-portfolio/scrnaseq/data/raw/GSE138852/GSE138852_counts.csv
Meta path: True /Users/oliviamohning/Documents/ds-portfolio/scrnaseq/data/raw/GSE138852/GSE138852_covariates.csv


### Load raw data and quick sanity checks

In [9]:
# Load data
counts_df = pd.read_csv(counts_path, index_col=0)
meta_df   = pd.read_csv(meta_path, index_col=0)

# Quick checks
print("Counts shape:", counts_df.shape)
print("Metadata shape:", meta_df.shape)

Counts shape: (10850, 13214)
Metadata shape: (13214, 5)


### Inspect first few rows of counts_df and meta_df

In [10]:
display(counts_df.head())
display(meta_df.head())

,AAACCTGGTAGAAAGG_AD5_AD6,AAACCTGGTAGCGATG_AD5_AD6,AAACCTGTCAGTCAGT_AD5_AD6,AAACCTGTCCAAACAC_AD5_AD6,AAACCTGTCCAGTATG_AD5_AD6,AAAGCAACATGGGAAC_AD5_AD6,AAAGCAAGTCGAATCT_AD5_AD6,AAAGCAAGTTTGTTGG_AD5_AD6,AAAGTAGGTAATCACC_AD5_AD6,AAAGTAGGTTCCACGG_AD5_AD6,...,TTTGGTTAGCCACGCT_AD1_AD2,TTTGGTTCAACTTGAC_AD1_AD2,TTTGGTTCAGCCTTTC_AD1_AD2,TTTGGTTCATCGGACC_AD1_AD2,TTTGGTTTCCCAGGTG_AD1_AD2,TTTGGTTTCCGTACAA_AD1_AD2,TTTGTCACAAGCCATT_AD1_AD2,TTTGTCAGTATAGGTA_AD1_AD2,TTTGTCATCCACTGGG_AD1_AD2,TTTGTCATCCGGGTGT_AD1_AD2
FO538757.2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
AP006222.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP5-857K21.4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
RP11-206L10.9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NOC2L,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


,oupSample.batchCond,oupSample.cellType,oupSample.cellType_batchCond,oupSample.subclustID,oupSample.subclustCond
AAACCTGGTAGAAAGG_AD5_AD6,AD,oligo,oligo_AD,o3,AD
AAACCTGGTAGCGATG_AD5_AD6,AD,oligo,oligo_AD,o3,AD
AAACCTGTCAGTCAGT_AD5_AD6,AD,oligo,oligo_AD,o3,AD
AAACCTGTCCAAACAC_AD5_AD6,AD,oligo,oligo_AD,o3,AD
AAACCTGTCCAGTATG_AD5_AD6,AD,oligo,oligo_AD,o3,AD


### Transpose rows and columns of counts_df so nuclei are rows and genes are columns, to match meta_df

In [11]:
counts_df = counts_df.T
display(counts_df.head())

,FO538757.2,AP006222.2,RP5-857K21.4,RP11-206L10.9,NOC2L,HES4,ISG15,AGRN,C1orf159,SDF4,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-CYB,AL592183.1,AC007325.4,AC007325.2
AAACCTGGTAGAAAGG_AD5_AD6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
AAACCTGGTAGCGATG_AD5_AD6,0,0,0,0,0,0,0,0,0,0,...,3,2,4,0,6,0,0,0,0,0
AAACCTGTCAGTCAGT_AD5_AD6,0,0,0,0,0,0,0,0,0,0,...,0,3,3,0,2,1,1,1,0,0
AAACCTGTCCAAACAC_AD5_AD6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
AAACCTGTCCAGTATG_AD5_AD6,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


### Keep only nuclei that exist in both counts and metadata

In [12]:
common_nuclei = counts_df.index.intersection(meta_df.index)
counts_df = counts_df.loc[common_nuclei]
meta_df = meta_df.loc[common_nuclei]

### Merge counts and metadata side by side, sanity checks, and previewing data

In [ ]:
# Merge meta_df and counts_df
merged_df = pd.concat([meta_df, counts_df], axis=1)

# Basic sanity checks
print("Merged shape (cells x [metadata + genes]):", merged_df.shape)
print("Metadata columns:", list(meta_df.columns))
print("Gene columns (first 5):", list(counts_df.columns[:5]))
print("First 3 cell IDs:", merged_df.index[:3].tolist())

# Previewing data
merged_df.head()

Merged shape (cells x [metadata + genes]): (13214, 10855)
Metadata columns: ['oupSample.batchCond', 'oupSample.cellType', 'oupSample.cellType_batchCond', 'oupSample.subclustID', 'oupSample.subclustCond']
Gene columns (first 5): ['FO538757.2', 'AP006222.2', 'RP5-857K21.4', 'RP11-206L10.9', 'NOC2L']
First 3 cell IDs: ['AAACCTGGTAGAAAGG_AD5_AD6', 'AAACCTGGTAGCGATG_AD5_AD6', 'AAACCTGTCAGTCAGT_AD5_AD6']


,oupSample.batchCond,oupSample.cellType,oupSample.cellType_batchCond,oupSample.subclustID,oupSample.subclustCond,FO538757.2,AP006222.2,RP5-857K21.4,RP11-206L10.9,NOC2L,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-CYB,AL592183.1,AC007325.4,AC007325.2
AAACCTGGTAGAAAGG_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
AAACCTGGTAGCGATG_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,3,2,4,0,6,0,0,0,0,0
AAACCTGTCAGTCAGT_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,3,3,0,2,1,1,1,0,0
AAACCTGTCCAAACAC_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
AAACCTGTCCAGTATG_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


### Renaming metadata columns for readability

In [22]:
merged_df.rename(columns={
    'oupSample.batchCond': 'condition',
    'oupSample.cellType': 'cell_type',
    'oupSample.cellType_batchCond': 'celltype_condition',
    'oupSample.subclustID': 'subcluster_id',
    'oupSample.subclustCond': 'subcluster_condition'
}, inplace=True)

merged_df.head()

,condition,cell_type,celltype_condition,subcluster_id,subcluster_condition,FO538757.2,AP006222.2,RP5-857K21.4,RP11-206L10.9,NOC2L,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-CYB,AL592183.1,AC007325.4,AC007325.2
AAACCTGGTAGAAAGG_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
AAACCTGGTAGCGATG_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,3,2,4,0,6,0,0,0,0,0
AAACCTGTCAGTCAGT_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,3,3,0,2,1,1,1,0,0
AAACCTGTCCAAACAC_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
AAACCTGTCCAGTATG_AD5_AD6,AD,oligo,oligo_AD,o3,AD,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


### Checking value counts of condition, cell_type, celltype_condition, subcluster_id, and subcluster_condition

In [35]:
merged_df['condition'] = merged_df['condition'].replace({'ct': 'Control', 'AD': 'Alzheimer'})
merged_df['condition'].value_counts()

condition
Alzheimer    6673
Control      6541
Name: count, dtype: int64

In [27]:
merged_df['cell_type'].value_counts()

cell_type
oligo      7432
astro      2171
OPC        1078
unID        925
neuron      656
mg          449
doublet     405
endo         98
Name: count, dtype: int64

In [29]:
merged_df['celltype_condition'].value_counts()

celltype_condition
oligo_AD      4655
oligo_ct      2777
astro_ct      1699
OPC_ct         899
unID_AD        614
astro_AD       472
neuron_ct      407
unID_ct        311
doublet_AD     295
mg_ct          277
neuron_AD      249
OPC_AD         179
mg_AD          172
doublet_ct     110
endo_ct         61
endo_AD         37
Name: count, dtype: int64

In [31]:
merged_df['subcluster_id'].value_counts()

subcluster_id
o2    2025
o6    1902
o1    1881
o5     827
o3     638
a5     600
a7     447
O4     415
a3     262
u7     256
O2     254
O3     238
a1     222
a2     219
a8     207
n5     192
u4     187
O1     171
m3     163
o4     159
u3     148
a4     146
n3     127
u6     121
d2     118
m2     104
n1     103
d4      94
d1      93
n4      90
n6      82
u2      79
m1      78
u1      75
a6      68
m5      66
d3      64
n2      62
u5      59
e1      55
e2      43
m4      38
d5      36
Name: count, dtype: int64

In [32]:
merged_df['subcluster_condition'].value_counts()

subcluster_condition
AD              6440
ct              6050
undetermined     724
Name: count, dtype: int64

### Compute average gene expression per cell type

In [34]:
celltype_means = merged_df.groupby('cell_type').mean(numeric_only=True)
celltype_means.head()

,FO538757.2,AP006222.2,RP5-857K21.4,RP11-206L10.9,NOC2L,HES4,ISG15,AGRN,C1orf159,SDF4,...,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-CYB,AL592183.1,AC007325.4,AC007325.2
cell_type,,,,,,,,,,,,,,,,,,,,,
OPC,0.129870,0.025974,0.192950,0.032468,0.021336,0.033395,0.056586,0.056586,0.022263,0.058442,...,0.194805,0.414657,0.185529,0.017625,0.397032,0.038961,0.218924,0.160482,0.046382,0.003711
astro,0.183326,0.047904,0.221557,0.032243,0.041916,0.094887,0.033625,0.053892,0.054353,0.071396,...,0.323814,0.614463,0.320129,0.049747,0.720405,0.074620,0.376785,0.122064,0.025795,0.085214
doublet,0.150617,0.037037,0.120988,0.019753,0.049383,0.024691,0.056790,0.076543,0.034568,0.091358,...,0.933333,1.733333,0.617284,0.083951,1.646914,0.143210,0.829630,0.133333,0.032099,0.012346
endo,0.132653,0.040816,0.132653,0.051020,0.040816,0.295918,0.081633,0.122449,0.020408,0.051020,...,0.714286,1.285714,0.816327,0.091837,1.132653,0.112245,0.734694,0.122449,0.030612,0.010204
mg,0.089087,0.026726,0.102450,0.022272,0.024499,0.002227,0.013363,0.013363,0.020045,0.040089,...,0.207127,0.398664,0.178174,0.028953,0.447661,0.028953,0.256125,0.095768,0.006682,0.000000


### Save cleaned dataset to proc_dir ("processed directory"), created above

In [36]:
processed_path = proc_dir / "01_processed_snrna_data.csv"
merged_df.to_csv(processed_path, index=True)
print(f"Saved processed dataset to: {processed_path}")


Saved processed dataset to: /Users/oliviamohning/Documents/ds-portfolio/scrnaseq/data/processed/01_processed_snrna_data.csv
